In [1]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.preprocessing import normalize

In [2]:
df = pd.read_csv('titanic.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C85
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,C123
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,NaN


The 'easy to work with' variables are: Survived, Pclass, Sex, SibSp & Parch
These are all numeric values within the same range; we can make dummies for the male/female bit.

In [3]:
sdf = df[['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch']]
sdf.dropna()
sdf.head(3)

,Survived,Pclass,Sex,SibSp,Parch
0,0,3,male,1,0
1,1,1,female,1,0
2,1,3,female,0,0


You use Dropna now because otherwise you'll lose half of your data (Cabin has a lot of NaNs)

In [4]:
dummies = pd.get_dummies(sdf['Sex'])
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [5]:
sdf = pd.concat([sdf, dummies], axis = 1)
sdf.head(3)

,Survived,Pclass,Sex,SibSp,Parch,female,male
0,0,3,male,1,0,0,1
1,1,1,female,1,0,1,0
2,1,3,female,0,0,1,0


In [6]:
sdf = sdf.drop(['male', 'Sex'], axis=1)
sdf.head(3)

,Survived,Pclass,SibSp,Parch,female
0,0,3,1,0,0
1,1,1,1,0,1
2,1,3,0,0,1


In [7]:
X = sdf[['Pclass', 'SibSp', 'Parch', 'female']]
X = normalize(X)
y = sdf['Survived']
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.3)

In [8]:
knn = KNeighborsClassifier(n_neighbors=3) 
knn = knn.fit(X_train_1, y_train_1)
knn.score(X_test_1, y_test_1)

0.7649253731343284

The K-nn score for the test set is 0.74


Calculate accuracy, precision and recall:

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
pred = knn.predict(X_test_1)
cm = confusion_matrix(y_test_1, pred)
cm

array([[128,  25],
       [ 38,  77]], dtype=int64)

In [11]:
cma = pd.DataFrame(cm, index=['not survived', 'survived'], columns = ['not survived pr', 'survived pr']) 
cma

,not survived pr,survived pr
not survived,128,25
survived,38,77


In [12]:
accuracy = (144 + 24) / (134 + 37 + 36 + 61)
accuracy

0.6268656716417911

In [13]:
precision = 144 / (144 + 24)
precision

0.8571428571428571

In [14]:
recall = 144 / (144 + 45)
recall

0.7619047619047619